In [ ]:
import pandas as pd
from datetime import datetime, timedelta
# import geopandas as gpd

In [ ]:
bbugs = pd.read_csv('./Data/bbugs_clean.csv')

In [ ]:
bbugs.columns

In [ ]:
grouped_events = bbugs.groupby(['YEAR_EVENT', 'WEEK_EVENT', 'No_QR'], as_index=False).agg({'NO_DECLARATION':'count', 'NBR_EXTERMIN':'sum'})
grouped_events = grouped_events.sort_values(['No_QR', 'YEAR_EVENT', 'WEEK_EVENT'])

In [ ]:
grouped_events.tail(5)

In [ ]:
grouped_events_copy = grouped_events.copy()
grouped_events_copy['No_QR'].unique()

In [ ]:
def get_id_week(date):
    return str(date.isocalendar()[0])+'-'+str(date.isocalendar()[1])

In [ ]:
# Le 3 janvier 2011 est un lundi
# le date id est de la forme YYYY-W ou YYYY-WW où YYYY est l'année grégorienne (qui peut être différente de l'année en cours)
# et WW la semaine grégorienne. Les deux sont obtenus avec isocalendar()
WEEK_ID = []
YEAR = []
MONTH = []
MONTH_NAME = []
WEEK = []
NO_QR = []
for no_qr in grouped_events_copy['No_QR'].unique():
    date = datetime(2011,1,3)
    while date < datetime(2019,6,30):
        curr_id_week = get_id_week(date)
        WEEK_ID.append(curr_id_week)
        YEAR.append(date.isocalendar()[0])        
        WEEK.append(date.isocalendar()[1])
        MONTH.append(date.month)
        MONTH_NAME.append(date.strftime("%B"))
        NO_QR.append(str(no_qr))
        date = date + timedelta(days=7)
df_final_bugs = pd.DataFrame()
df_final_bugs['WEEK_ID'] = WEEK_ID
df_final_bugs['YEAR'] = YEAR
df_final_bugs['WEEK'] = WEEK
df_final_bugs['NO_QR'] = NO_QR
df_final_bugs['MONTH'] = MONTH
df_final_bugs['MONTH_NAME'] = MONTH_NAME
df_final_bugs

In [ ]:
# On parcourt les semaines et on ajoute le nombre d'extermintation
NB_EXTERMINATIONS = []
for index, row in df_final_bugs.iterrows():
    curr_week = row['WEEK']
    curr_year = row['YEAR']
    curr_no_qr = row['NO_QR']
    year_evts = grouped_events_copy[grouped_events_copy['YEAR_EVENT'] == curr_year]
    week_evts = year_evts[year_evts['WEEK_EVENT'] == curr_week]
    no_qr_evts = week_evts[week_evts['No_QR'] == curr_no_qr]
    NB_EXTERMINATIONS.append(no_qr_evts['NBR_EXTERMIN'].sum())

# On récupère le nombre d'exterminations cumulées sur la semaine actuelle et la dernière semaine
NB_EXTERMINATIONS_2 = [0] + [NB_EXTERMINATIONS[i] + NB_EXTERMINATIONS[i+1] for i in range(len(NB_EXTERMINATIONS) - 1)]
NB_EXTERMINATIONS_3 = [0]*2 + [NB_EXTERMINATIONS[i] + NB_EXTERMINATIONS[i+1] + NB_EXTERMINATIONS[i+2] for i in range(len(NB_EXTERMINATIONS) - 2)]
NB_EXTERMINATIONS_4 = [0]*3 + [NB_EXTERMINATIONS[i] + NB_EXTERMINATIONS[i+1] + NB_EXTERMINATIONS[i+2] + NB_EXTERMINATIONS[i+3] for i in range(len(NB_EXTERMINATIONS) - 3)]

In [ ]:
df_final_bugs['NB_EXTERMINATIONS'] = NB_EXTERMINATIONS
df_final_bugs['NB_EXTERMINATIONS_2'] = NB_EXTERMINATIONS_2
df_final_bugs['NB_EXTERMINATIONS_3'] = NB_EXTERMINATIONS_3
df_final_bugs['NB_EXTERMINATIONS_4'] = NB_EXTERMINATIONS_4

In [ ]:
# Ajout du label. On regarde le nombre d'exterminations qu'il y aura dans les 3 mois à venir
LABEL_1mois = [sum(NB_EXTERMINATIONS[i:i+4]) for i in range(len(NB_EXTERMINATIONS))]
LABEL_3mois = [sum(NB_EXTERMINATIONS[i:i+12]) for i in range(len(NB_EXTERMINATIONS))]
df_final_bugs['LABEL_1mois'] = LABEL_1mois
df_final_bugs['LABEL_3mois'] = LABEL_3mois
df_final_bugs

In [ ]:
df_final_bugs.to_csv('bbugs_final.csv', index=False)

In [ ]:
# events_b['EXTERMIN_WEEK-1'] = 0
# events_b['EXTERMIN_WEEK-2'] = 0
# events_b['EXTERMIN_WEEK-3'] = 0
# events_b['EXTERMIN_MONTH-1'] = 0
# events_b['NUM_WEEKS_SINCE_LAST'] = 0

In [ ]:
# week = 1
# week_offset = 2

In [ ]:
# def week_year_counter(week, year, week_offset):
#     nb_weeks_in_year
#     week_ = (week - week_offset - 1) % 54 + 1
#     if week != (week - week_offset):
#         year_ = year - 1
#     else:
#         year_ = year
#     return(week_, year_)

In [ ]:
# def get_condition(week, year, events):
#     return ((events['WEEK_EVENT'] == week) & (events['YEAR_EVENT'] == year ))

In [ ]:
# for q in events_b['No_QR'].unique():
#     events_bb = events_b[events_b['No_QR'] == q]
#     length = len(events_bb)
#     for i, rows in events_bb.iterrows():
#         #print(str((i/length)*100) + " percent complete         \r")
#         week_1, year_1 = week_year_counter(rows['WEEK_EVENT'], rows['YEAR_EVENT'], 1)
#         week_2, year_2 = week_year_counter(rows['WEEK_EVENT'], rows['YEAR_EVENT'], 2)
#         week_3, year_3 = week_year_counter(rows['WEEK_EVENT'], rows['YEAR_EVENT'], 3)
#         week_4, year_4 = week_year_counter(rows['WEEK_EVENT'], rows['YEAR_EVENT'], 4)
#         print(i)
#         cond1 = events_bb[get_condition(week_1, year_1, events_bb)]['NBR_EXTERMIN']
#         cond2 = events_bb[get_condition(week_2, year_2, events_bb)]['NBR_EXTERMIN']
#         cond3 = events_bb[get_condition(week_3, year_3, events_bb)]['NBR_EXTERMIN']
#         cond4 = events_bb[get_condition(week_4, year_4, events_bb)]['NBR_EXTERMIN']
#         if not cond1.empty:
#             print(cond1)
#             events_b['EXTERMIN_WEEK-1'].iloc[i] += cond1.values[0]
#             events_b['EXTERMIN_WEEK-2'].iloc[i] += cond1.values[0]
#             events_b['EXTERMIN_WEEK-3'].iloc[i] += cond1.values[0]
#             events_b['EXTERMIN_MONTH-1'].iloc[i] += cond1.values[0]
#         if not cond2.empty:
#             events_b['EXTERMIN_WEEK-2'].iloc[i] += cond2.values[0]
#             events_b['EXTERMIN_WEEK-3'].iloc[i] += cond2.values[0]
#             events_b['EXTERMIN_MONTH-1'].iloc[i]+= cond2.values[0]
#         if not cond3.empty:
#             events_b['EXTERMIN_WEEK-3'].iloc[i] += cond3.values[0]
#             events_b['EXTERMIN_MONTH-1'].iloc[i]+= cond3.values[0]
#         if not cond4.empty:
#             events_b['EXTERMIN_MONTH-1'].iloc[i]+= cond4.values[0]
      #  events_b['EXTERMIN_WEEK-2'].iloc[i] = events_bb[get_condition(week_2, year_2, events_bb)]['NBR_EXTERMIN'] + events_b['EXTERMIN_WEEK-1'].iloc[i]
       # events_b['EXTERMIN_WEEK-3'].iloc[i] = events_bb[get_condition(week_3, year_3, events_bb)]['NBR_EXTERMIN'] + events_b['EXTERMIN_WEEK-2'].iloc[i]
       # events_b['EXTERMIN_MONTH-1'].iloc[i] = events_bb[get_condition(week_4, year_4, events_bb)]['NBR_EXTERMIN'] + events_b['EXTERMIN_WEEK-3'].iloc[i]
       # events_b['NUM_WEEKS_SINCE_LAST'].iloc[i] = 

    